# Algoritmos Minimax e Alfabeta

In [ ]:
from collections import namedtuple
import random

from utils import argmax
from canvas import Canvas

infinity = float('inf')
GameState = namedtuple('GameState', 'to_move, utility, board, moves')

def minimax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minimax_decision:
    return argmax(game.actions(state),
                  key=lambda a: min_value(game.result(state, a)))


def alphabeta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search:
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


def alphabeta_cutoff_search(state, game, d=1, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""
    eval_fn = game.eval_function
    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = -infinity
        for a in game.actions(state):
            st = game.result(state, a)
            #game.display(st)
            v = max(v, min_value(st, alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state, player)
        v = infinity
        for a in game.actions(state):
            st = game.result(state, a)
            #game.display(st)
            v = min(v, max_value(st, alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alphabeta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state, player : game.utility(state, player))
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        st = game.result(state, a)
        #game.display(st)
        v = min_value(st, best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

## Definição de 4 tipos de jogadores
<b>query_player</b>: oponente humano <br>
<b>random_player</b>: agente que joga de forma aleatória <br>
<b>alphabeta_player</b>: agente que joga utilizando o minimax alfabeta <br>
<b>alphabetacutoff_player</b>: agente que joga utilizando o minimax alfabeta com função de avaliação

In [ ]:
def query_player(game, state):
    """Make a move by querying standard input."""
    print("current state:")
    game.display(state)
    print("available moves: {}".format(game.actions(state)))
    print("")
    move_string = input('Your move? ')
    try:
        move = eval(move_string)
    except NameError:
        move = move_string
    return move


def random_player(game, state):
    """A player that chooses a legal move at random."""
    return random.choice(game.actions(state))

def alphabeta_player(game, state):
    return alphabeta_search(state, game)

def alphabetacutoff_player(game, state):
    return alphabeta_cutoff_search(state, game)

## Definição da classe que representa um jogo

In [ ]:
class Game:
    """A game has a utility for each
    state and a terminal test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                #self.display(state)
                if self.terminal_test(state):
                    #self.display(state)
                    return self.utility(state, self.to_move(self.initial))

## JOGO: Jogo da Velha

In [ ]:
class TicTacToe(Game):
    """Play TicTacToe on an h x v board, with Max (first player) playing 'X'.
    A state has the player to move, a cached utility, a list of moves in
    the form of a list of (x, y) positions, and a board, in the form of
    a dict of {(x, y): Player} entries, where Player is 'X' or 'O'."""

    def __init__(self, h=3, v=3, k=3):
        self.h = h
        self.v = v
        self.k = k
        moves = [(x, y) for x in range(1, h + 1)
                 for y in range(1, v + 1)]
        self.initial = GameState(to_move='X', utility=0, board={}, moves=moves)

    def actions(self, state):
        """Legal moves are any square not yet taken."""
        return state.moves

    def result(self, state, move):
        if move not in state.moves:
            return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                             utility=self.compute_utility(state.board, move, state.to_move),
                             board=state.board, moves=state.moves)
        board = state.board.copy()
        board[move] = state.to_move
        moves = list(state.moves)
        moves.remove(move)
        return GameState(to_move=('O' if state.to_move == 'X' else 'X'),
                         utility=self.compute_utility(board, move, state.to_move),
                         board=board, moves=moves)

    def utility(self, state, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return state.utility if player == 'X' else -state.utility  
    
    def eval_function(self, state, player):
            v = 0            
            oponent = 'O' if player == 'X' else 'X'
            
            for x in range(1, 4):
                for y in range(1, 4):
                    if state.board.get((x, y)) == player:
                        v += 2 if ((x+y) % 2) == 1 else 3 if (not (x == 2)) else 4
            for x in range(1, 4):
                for y in range(1, 4):
                    if state.board.get((x, y)) == oponent:
                        v -= 2 if ((x+y) % 2) == 1 else 3 if (not (x == 2)) else 4
                        
            u = self.utility(state, player)
            return u if not u == 0 else v  

    def terminal_test(self, state):
        """A state is terminal if it is won or there are no empty squares."""
        return state.utility != 0 or len(state.moves) == 0

    def display(self, state):
        board = state.board
        for x in range(1, self.h + 1):
            for y in range(1, self.v + 1):
                print(board.get((x, y), '.'), end=' ')
            print()
        print()    

    def compute_utility(self, board, move, player):
        """If 'X' wins with this move, return 1; if 'O' wins return -1; else return 0."""
        if (self.k_in_row(board, move, player, (0, 1)) or
                self.k_in_row(board, move, player, (1, 0)) or
                self.k_in_row(board, move, player, (1, -1)) or
                self.k_in_row(board, move, player, (1, 1))):
            return +20 if player == 'X' else -20
        else:
            return 0

    def k_in_row(self, board, move, player, delta_x_y):
        """Return true if there is a line through move on board for player."""
        (delta_x, delta_y) = delta_x_y
        x, y = move
        n = 0  # n is number of moves in row
        while board.get((x, y)) == player:
            n += 1
            x, y = x + delta_x, y + delta_y
        x, y = move
        while board.get((x, y)) == player:
            n += 1
            x, y = x - delta_x, y - delta_y
        n -= 1  # Because we counted move itself twice
        return n >= self.k

## Ultimate Tic-Tac-Toe

In [ ]:
## variaveis iniciais e de configuração
## win_resul armazena as posições de todas as vitorias
## last_move indica em qual quadrado deverá ser a proxima jogada(-1 significa qualquer lugar)
win_resul = [(0,4,8),(2,4,6)]
win_resul += [(i, i+3, i+6) for i in range(3)]
win_resul += [(i*3, i*3+1, i*3+2) for i in range(3)]
last_move = -1

## cria e preenche o tabuleiro
def create_board():
    board = []
    square = ['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F']
    for i in range(9):
        board.append(square.copy())
    return board

## imprime o tabuleiro
def print_board(board):
    for h in range(3):
        print("-"*25)
        for i in range(3):
            line = "| "
            for j in range(3):
                line += board[j+h*3][i*3] + " " + board[j+h*3][i*3+1] + " " + board[j+h*3][i*3+2] + " | "
            print(line)
    print("-"*25)

## verifica se há vencedor num dado quadrado
def small_win(square):
    for i in win_resul:
        (x, y, z) = i
        square_x = square[x]
        if(square_x == square[y] == square[z]) and (square[x] != 'L'):
            return square_x
    return 'L'

## verifica se há vencedor no tabuleiro
def big_win(board):
    for i in win_resul:
        (x, y, z) = i
        small_x = small_win(board[x])
        if(small_x == small_win(board[y]) == small_win(board[z])) and (small_win(board[x]) != 'L'):
            return small_x
    return 'L'

## recebe o tabuleiro, as posições da jogada e o jogador, e retorna se a jogada é válida
## x é o numero do quadrado a ser jogado e y a posição nele
def player_move(board, x, y, player):
    if(board[x][y] == 'L'):
        return False, board
    else:
        last_move = y ## atualiza o ultimo movimento pra sabermos qual será o próximo quadrado a ser jogado
        board[x][y] = player
        return True, board

## verifica se o quadrado está todo preenchido e retorna um boolean correspondente
def full_board(board):
    for i in board[last_move]:
        if(i=='L'):
            return False
    return True

## verifica se no quadrado indicado pela ultima jogada há vitoria ou empate, caso haja last_move passa a ser -1
def next_move(board):
    target_square = board[last_move]
    if(small_win(target_square) or full_board(target_square)):
        last_move = -1